# Problem description

This notebook is solution for the task under the project Physics-Informed Neural Network Diffusion Equation (PINNDE) by GENIE @ ML4SCI under GSoC'25.

In this notebook we will implement a PINN to solve the following damped harmonic oscillator differential equation using various approaches

### Differential Equation:
The given second-order differential equation is:
<div align="center">

#### $\frac{d^2x}{dz^2} + 2\xi \frac{dx}{dz} + x = 0$

</div>

with the initial conditions:

<div align="center">
    
#### $x(0) = x_0, \quad \frac{dx}{dz}(0) = v_0, \quad \text{where} \quad x_0 =$ $0.7, \quad v_0 = 1.2.$

</div>

The damping ratio ranges as:

<div align="center">

#### $\xi \in [0.1, 0.4].$

</div>

The solution is sought over the domain:

<div align="center">

#### $z \in [0, 20].$

</div>


In [1]:
# necessary imports and configs
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
device = 'cuda' if torch.cuda.is_available() else 'cpu'

x0 = 0.7
v0 = 1.2
z_min = 0.0
z_max = 20
xi_min = 0.1
xi_max = 0.4

## Generating correct physical data for evaluating approaches

Since this equation can be solved, I can benchmark proposed models using the analytical solutio of the equation, hence I will curate an `eval_dataset`.

### Solution of the equation

<div align="center">

#### $x(z, \xi) = e^{-\xi z} \left( 0.7 \cos(\omega_d z) + \frac{1.2 + 0.7\xi}{\omega_d} \sin(\omega_d z) \right),$

</div>

wher,

<div align="center">

#### $\omega_d = \sqrt{1 - \xi^2}.$

</div>

#### NOTE : This Dataset will contain various values of $z$, $\xi$ and their corresponding $x$ values and will only be used for evaluating PINNs

In [2]:
def generate_eval_dataset(num_samples=100, save_path="./"):
    # Randomly sample z and xi values within the given range
    z_random = np.random.uniform(z_min, z_max, num_samples)
    xi_random = np.random.uniform(xi_min, xi_max, num_samples)
    
    # Compute corresponding x values based on the analytical solution
    data_random = []
    
    for z, xi in zip(z_random, xi_random):
        omega_d = np.sqrt(1 - xi**2)  
        C1 = x0  
        C2 = (v0 + xi * x0) / omega_d  
    
        x = np.exp(-xi * z) * (C1 * np.cos(omega_d * z) + C2 * np.sin(omega_d * z))
        data_random.append([z, xi, x])
    
    df_random = pd.DataFrame(data_random, columns=["z", "xi", "x"])
    
    dataset_random_path = save_path + "pinn_de_dataset.csv"
    df_random.to_csv(dataset_random_path, index=False)

    return df_random


In [3]:
# generate data
eval_dataset = generate_eval_dataset()

# Simple PINN (Approach 1)

### 1. Training Data Generation
   * A grid of $z$ values ($100$ points) and $\xi$ values ($10$ points) is created.
   * The dataset is randomized by shuffling rows to improve training.
   * Gradients are tracked for $z$ and $\xi$, enabling automatic differentiation.

In [4]:
z_train = torch.linspace(z_min, z_max, 100).view(-1, 1)  # Domain
xi_train = torch.linspace(xi_min, xi_max, 10).view(-1, 1)  # Damping ratios

z_grid, xi_grid = torch.meshgrid(z_train.squeeze(), xi_train.squeeze(), indexing='ij')
z_train_flat, xi_train_flat = z_grid.reshape(-1, 1), xi_grid.reshape(-1, 1)

train_data = torch.cat((z_train_flat, xi_train_flat), dim=1)
train_data = train_data[torch.randperm(train_data.shape[0])]
z_train_flat, xi_train_flat = train_data[:, 0].view(-1, 1), train_data[:, 1].view(-1, 1)

z_train_flat.requires_grad = True
xi_train_flat.requires_grad = True

z_train_flat = z_train_flat.to(device)
xi_train_flat = xi_train_flat.to(device)

print(f"Length of dataset : {z_train_flat.shape[0]}")

Length of dataset : 1000


### 2. Define PINN architecture
* Input Layer: Takes two inputs, $z$ (position) and $\xi$ (damping ratio).
* Hidden Layer: Fully connected (Linear) with $128$ neurons and Tanh activation.
* Output Layer: A single neuron (Linear activation) predicts $x(z,\xi).$

In [5]:
class PINN(nn.Module):
    def __init__(self):
        super(PINN, self).__init__()
        self.hidden = nn.Sequential(
            nn.Linear(2, 128),
            nn.Tanh(),
            nn.Linear(128, 128),
            nn.Tanh(),
            nn.Linear(128, 1)
        )
    
    def forward(self, z, xi):
        inputs = torch.cat((z, xi), dim=1)
        return self.hidden(inputs)

### 3. Defining Loss Functions
* Total training loss is defined as follows
  

    
<div align="center"> $\mathcal{L} = \mathcal{L}_{\text{residual}} + \mathcal{L}_{\text{IC}}$ </div>



* Where residual loss is calculated as follows


 <div align="center"> $\mathcal{L}_{\text{residual}} = \frac{1}{N} \sum_{i=1}^{N} \left( \frac{d^2x}{dz^2} + 2\xi \frac{dx}{dz} + x \right)^2$ </div>


    
* Initial conditions are enforced as follows


<div align="center"> $\mathcal{L}_{\text{IC}} = \lambda_1 \left( x(0) - x_0 \right)^2 + \lambda_2 \left( \frac{dx}{dz}(0) - v_0 \right)^2$ </div>



* L2 Loss to evaluate the training on `eval_dataset`

<div align="center"> $L_2 \text{ error} = \frac{||x_{\text{PINN}} - x_{\text{exact}}||_2}{||x_{\text{exact}}||_2}$ </div>
 




In [6]:
lambda1 = 1000
lambda2 = 1000
def loss_fn():
    x_pred = model(z_train_flat, xi_train_flat)
    x_z = torch.autograd.grad(x_pred, z_train_flat, torch.ones_like(x_pred), create_graph=True)[0]
    x_zz = torch.autograd.grad(x_z, z_train_flat, torch.ones_like(x_z), create_graph=True)[0]
    
    residual = x_zz + 2 * xi_train_flat * x_z + x_pred
    
    # Initial conditions
    z0 = torch.tensor([[z_min]], requires_grad=True, device=device)
    xi0 = torch.tensor([[xi_min]], requires_grad=True, device=device)
    x0_pred = model(z0, xi0)
    v0_pred = torch.autograd.grad(x0_pred, z0, torch.ones_like(x0_pred), create_graph=True)[0]
    ic_loss = lambda1*(x0_pred - x0) ** 2 + lambda2*(v0_pred - v0) ** 2
    
    return torch.mean(residual**2) + torch.mean(ic_loss)

def compute_L2_error(x_pinn, x_exact):
    error_norm = torch.norm(x_pinn - x_exact, p=2)
    exact_norm = torch.norm(x_exact, p=2)
    L2_error = (error_norm / exact_norm).item()
    return L2_error

### 4. Training Strategy
* Training at the beginning using `AdamW` optimizer
* Using scheduler `ReduceLROnPlateau` with initial learning rate $0.001$
* Further training using `LBFGS` optimizer for more accurate results

In [7]:
# Initializing 
epochs = 10_000
lr=1e-3
model = PINN().to(device)
optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
z_test = torch.tensor(eval_dataset['z'], 
                      dtype=torch.float32).reshape((len(eval_dataset),1)).to(device)
xi_test = torch.tensor(eval_dataset['xi'], 
                       dtype=torch.float32).reshape((len(eval_dataset),1)).to(device)
x_exact = torch.tensor(eval_dataset['x'], 
                       dtype = torch.float32).reshape(len(eval_dataset), 1).to(device)

In [8]:
# Training loop
train_losses = []
l2_losses = []
best_l2 = float('inf')
for epoch in range(epochs):
    optimizer.zero_grad()
    loss = loss_fn()
    loss.backward()
    optimizer.step()
    scheduler.step()
    with torch.no_grad():
        x_pinn = model(z_test, xi_test)
        l2_error = compute_L2_error(x_pinn, x_exact)
        l2_losses.append(l2_error/z_test.shape[0])
    train_losses.append(loss.item()/z_train_flat.shape[0])
    if l2_error < best_l2:
        best_l2 = l2_error
        torch.save({
            'model_dict':model.state_dict()
        }, 'Simple_PINN.pth')
    
    if epoch % 1000 == 0:
        print(f"Epoch {epoch}: Loss = {loss.item():.6f}, L2 Loss = {l2_error:.6f}")

Epoch 0: Loss = 2119.564941, L2 Loss = 1.330985
Epoch 1000: Loss = 0.139734, L2 Loss = 0.923533
Epoch 2000: Loss = 0.064373, L2 Loss = 0.646308
Epoch 3000: Loss = 0.037214, L2 Loss = 0.528642
Epoch 4000: Loss = 0.028658, L2 Loss = 0.514332
Epoch 5000: Loss = 0.023829, L2 Loss = 0.513687
Epoch 6000: Loss = 0.019935, L2 Loss = 0.513199
Epoch 7000: Loss = 0.017740, L2 Loss = 0.511491
Epoch 8000: Loss = 0.016548, L2 Loss = 0.508887
Epoch 9000: Loss = 0.015984, L2 Loss = 0.506966


In [9]:
# Loding best model state and further training using LBFGS
check_point = torch.load("Simple_PINN.pth")
model = PINN().to(device)
model.load_state_dict(check_point['model_dict'])
epochs = 500
lr=1e-4
optimizer = optim.LBFGS(model.parameters(), lr=lr, line_search_fn="strong_wolfe")
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)

In [10]:
# closure function for LBFGS
def closure():
    optimizer.zero_grad()
    loss = loss_fn()
    loss.backward()
    return loss

In [11]:
#Training loop 
best_l2 = float('inf')
for epoch in range(epochs):
    loss = optimizer.step(closure)
    with torch.no_grad():
        x_pinn = model(z_test, xi_test)
        l2_error = compute_L2_error(x_pinn, x_exact)
        l2_losses.append(l2_error/z_test.shape[0])
    train_losses.append(loss.item()/z_train_flat.shape[0])
    if l2_error < best_l2:
        best_l2 = l2_error
        torch.save({
            'model_dict':model.state_dict()
        }, 'Simple_PINN.pth')
    if epoch % 50 == 0:
        print(f"Epoch {epoch}: Loss = {loss.item():.6f}, L2 Loss = {l2_error:.6f}")
        

Epoch 0: Loss = 0.015870, L2 Loss = 0.506523
Epoch 50: Loss = 0.015870, L2 Loss = 0.506523
Epoch 100: Loss = 0.015870, L2 Loss = 0.506523
Epoch 150: Loss = 0.015870, L2 Loss = 0.506523
Epoch 200: Loss = 0.015870, L2 Loss = 0.506523
Epoch 250: Loss = 0.015870, L2 Loss = 0.506523
Epoch 300: Loss = 0.015870, L2 Loss = 0.506523
Epoch 350: Loss = 0.015870, L2 Loss = 0.506523
Epoch 400: Loss = 0.015870, L2 Loss = 0.506523
Epoch 450: Loss = 0.015870, L2 Loss = 0.506523


In [12]:
print(f"Best L2 score achieved through Simple PINN {best_l2}")

Best L2 score achieved through Simple PINN 0.5065234303474426


# PINN using Fourier features (Approach 2)
### 1. Data Generation
* Same as previous models

### 2. Define PINNF architecture
* Fourier Feature Mapping: Uses 16 Fourier features with a scaling factor $σ=1.0$ to enhance input representation.
* Fixed Fourier Features: The B matrix (random frequencies) is initialized and not trainable, ensuring stable feature transformation.
* Expanded Input Dimension: The input $(z,ξ)$ is expanded to $2+2×16=34 $dimensions using sine and cosine projections.
* Deep Network Architecture: 7 fully connected layers with Tanh activation.
* Layer sizes:`input`$→128→256→512→512→256→128→$`output`
* Projection to Frequency Domain: Transforms inputs using sine and cosine functions before passing through the network.
* High Expressivity: Fourier features help the network learn high-frequency components, improving accuracy for complex functions.

### 3. Training Strategy
* Same as before

In [13]:
class PINNF(nn.Module):
    def __init__(self, input_dim=2, hidden_dim=128, output_dim=1, n_fourier_features=16, sigma=1.0):
        super(PINNF, self).__init__()
        
        # Fourier feature parameters
        self.n_fourier_features = n_fourier_features
        self.sigma = sigma
        self.B = torch.randn(input_dim, n_fourier_features) * sigma
        self.B = nn.Parameter(self.B, requires_grad=False)  # Fixed random Fourier features
        
        # Calculate the expanded input dimension after applying Fourier features
        expanded_dim = input_dim + 2 * n_fourier_features
        
        # Network layers
        self.hidden = nn.Sequential(
            nn.Linear(expanded_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, 256),
            nn.Tanh(),
            nn.Linear(256, 512),
            nn.Tanh(),
            nn.Linear(512, 512),
            nn.Tanh(),
            nn.Linear(512, 256),
            nn.Tanh(),
            nn.Linear(256, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, output_dim)
        )
    
    def fourier_features(self, x):
        # x shape: [batch_size, input_dim]
        # Project input to the frequency domain
        projection = torch.matmul(x, self.B)  # [batch_size, n_fourier_features]
        
        # Apply sine and cosine to get Fourier features
        sin_features = torch.sin(projection)
        cos_features = torch.cos(projection)
        
        # Concatenate original input with Fourier features
        return torch.cat([x, sin_features, cos_features], dim=1)
    
    def forward(self, z, xi):
        # Combine spatial coordinates
        inputs = torch.cat((z, xi), dim=1)
        
        # Apply Fourier feature mapping
        inputs_mapped = self.fourier_features(inputs)
        
        # Pass through the network
        return self.hidden(inputs_mapped)

In [14]:
# initializing models
epochs = 10_000
lr=1e-3
model = PINNF().to(device)
optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)

In [15]:
train_losses = []
l2_losses = []
best_l2 = float('inf')
for epoch in range(epochs):
    optimizer.zero_grad()
    loss = loss_fn()
    loss.backward()
    #torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()
    scheduler.step(loss)
    with torch.no_grad():
        x_pinn = model(z_test, xi_test)
        l2_error = compute_L2_error(x_pinn, x_exact)
        l2_losses.append(l2_error/z_test.shape[0])
    train_losses.append(loss.item()/z_train_flat.shape[0])
    if l2_error < best_l2:
        best_l2 = l2_error
        torch.save({
            'model_dict':model.state_dict()
        }, 'PINN_Fourier.pth')
    
    if epoch % 1000 == 0:
        print(f"Epoch {epoch}: Loss = {loss.item():.6f}, L2 Loss = {l2_error:.6f}")

Epoch 0: Loss = 2020.476807, L2 Loss = 0.862043
Epoch 1000: Loss = 0.053909, L2 Loss = 0.452719
Epoch 2000: Loss = 0.013367, L2 Loss = 0.376600
Epoch 3000: Loss = 0.010999, L2 Loss = 0.338796
Epoch 4000: Loss = 2.232193, L2 Loss = 0.317781
Epoch 5000: Loss = 0.015991, L2 Loss = 0.295341
Epoch 6000: Loss = 0.012850, L2 Loss = 0.290657
Epoch 7000: Loss = 0.534728, L2 Loss = 0.342926
Epoch 8000: Loss = 0.665913, L2 Loss = 0.291897
Epoch 9000: Loss = 0.031646, L2 Loss = 0.261812


In [16]:
check_point = torch.load("PINN_Fourier.pth")
model = PINNF().to(device)
model.load_state_dict(check_point['model_dict'])
epochs = 500
lr=1e-4
optimizer = optim.LBFGS(model.parameters(), lr=lr, line_search_fn="strong_wolfe")
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)

In [17]:
for epoch in range(epochs):
    loss = optimizer.step(closure)
    with torch.no_grad():
        x_pinn = model(z_test, xi_test)
        l2_error = compute_L2_error(x_pinn, x_exact)
        l2_losses.append(l2_error/z_test.shape[0])
    train_losses.append(loss.item()/z_train_flat.shape[0])
    if l2_error < best_l2:
        best_l2 = l2_error
        torch.save({
            'model_dict':model.state_dict()
        }, 'PINN_Fourier.pth')
    if epoch % 50 == 0:
        print(f"Epoch {epoch}: Loss = {loss.item():.6f}, L2 Loss = {l2_error:.6f}")

Epoch 0: Loss = 1.932029, L2 Loss = 0.264174
Epoch 50: Loss = 0.012619, L2 Loss = 0.264406
Epoch 100: Loss = 0.012619, L2 Loss = 0.264406
Epoch 150: Loss = 0.012619, L2 Loss = 0.264406
Epoch 200: Loss = 0.012619, L2 Loss = 0.264406
Epoch 250: Loss = 0.012619, L2 Loss = 0.264406
Epoch 300: Loss = 0.012619, L2 Loss = 0.264406
Epoch 350: Loss = 0.012619, L2 Loss = 0.264406
Epoch 400: Loss = 0.012619, L2 Loss = 0.264406
Epoch 450: Loss = 0.012619, L2 Loss = 0.264406


In [18]:
print(f"Best L2 score using Fourier features: {best_l2}")

Best L2 score using Fourier features: 0.25400981307029724


# Final Comments
* Significant Improvement with using fourier features as they are suitable for solving PDEs with oscillatory behaviour
* Implemented physics-informed custom loss function
* L2 loss is consistent with custom loss i.e. as custom loss decreases, L2 loss on analytical solution also decreases
* Evaluated on exact solutions hence provides clear picture